<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/R_Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data
# #Training
# # #Labels
# # #Images
# #Validation
# # #Labels
# # #Images

Install Packages

In [1]:
install.packages(c('raster','sf','fasterize'))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘sp’, ‘Rcpp’, ‘terra’, ‘classInt’, ‘s2’, ‘units’, ‘wk’, ‘RcppArmadillo’




In [2]:
install.packages("googledrive")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
install.packages("devtools")

devtools:::install_github("gearslaboratory/gdalUtils")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘shiny’, ‘downlit’, ‘htmlwidgets’, ‘miniUI’, ‘pkgdown’, ‘profvis’, ‘urlchecker’





R.methodsS3 (NA -> 1.8.2 ) [CRAN]
R.oo        (NA -> 1.26.0) [CRAN]
iterators   (NA -> 1.0.14) [CRAN]
R.utils     (NA -> 2.12.3) [CRAN]
foreach     (NA -> 1.5.2 ) [CRAN]


Skipping 1 packages not available: rgdal

Installing 6 packages: R.methodsS3, R.oo, iterators, rgdal, R.utils, foreach

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘rgdal’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmp2NoMLs/remotes3b64428284f/gearslaboratory-gdalUtils-8872aaf/DESCRIPTION’ ... OK
* preparing ‘gdalUtils’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘gdalUtils_2.0.3.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘/tmp/Rtmp2NoMLs/file3b665ca74a3/gdalUtils_2.0.3.2.tar.gz’ had non-zero exit status”


Load Libraries

In [4]:
library(raster)
library(sf)
library(fasterize)
library(magrittr)
library(googledrive)
library(parallel)
library(data.table)
library(stringr)
# library(gdalUtils)

Loading required package: sp

Linking to GEOS 3.11.1, GDAL 3.6.4, PROJ 9.1.1; sf_use_s2() is TRUE


Attaching package: ‘fasterize’


The following object is masked from ‘package:graphics’:

    plot


The following object is masked from ‘package:base’:

    plot



Attaching package: ‘magrittr’


The following object is masked from ‘package:raster’:

    extract



Attaching package: ‘data.table’


The following object is masked from ‘package:raster’:

    shift




Connect to Google Drive

In [5]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=16032e1c29145f8d4ae0d48351b0a3fc&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQWVhWVNIQmIwN1lFTUdTbGxLLXR4dXJMcjZYV0otWjFRRnIwVUJzNVh3TTk0U1hjU05HZzh0UW50dlFJNmlNelc5RnZ2ZyIsInN0YXRlIjoiMTYwMzJlMWMyOTE0NWY4ZDRhZTBkNDgzNTFiMGEzZmMifQ==


In [6]:
dir.create('input_data')

In [7]:
# https://drive.google.com/file/d/1thj0vGXN0xgXxI_XDY-hagp-gWHZCdaS/view?usp=drive_link
drive_download(file = as_id('1thj0vGXN0xgXxI_XDY-hagp-gWHZCdaS'), path =  '/content/input_data/naip_index.zip', overwrite = FALSE)
unzip(zipfile ='/content/input_data/naip_index.zip', exdir = "/content/input_data/naip_index/")
naip_index_gpkg = st_read('/content/input_data/naip_index/tileindex_HI_NAIP_2021.shp')

File downloaded:

• tileindex_HI_NAIP_2021.zip <id: 1thj0vGXN0xgXxI_XDY-hagp-gWHZCdaS>

Saved locally as:

• /content/input_data/naip_index.zip



Reading layer `tileindex_HI_NAIP_2021' from data source 
  `/content/input_data/naip_index/tileindex_HI_NAIP_2021.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 611 features and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -160.3144 ymin: 18.8732 xmax: -154.7481 ymax: 22.25184
Geodetic CRS:  NAD83


In [8]:
# NWI wetlands for all islands
# https://drive.google.com/file/d/1ozE_Ldpio1LJaH2WS9r9hqRAIBDV8TDF/view?usp=drive_link


# NWI wetlands for the islands in UTM 4 (kauai, oahu, molokai, maui)
# https://drive.google.com/file/d/1uKY_cwL8P_bir8T4VY3f-gKs4ZQQyYk9/view?usp=drive_link
drive_download(file = as_id('1uKY_cwL8P_bir8T4VY3f-gKs4ZQQyYk9'), path =  '/content/input_data/NWI_wetlands.gpkg', overwrite = TRUE)
nwi_gpkg = st_read('/content/input_data/NWI_wetlands.gpkg')

File downloaded:

• nwi_wetlands_4.gpkg <id: 1uKY_cwL8P_bir8T4VY3f-gKs4ZQQyYk9>

Saved locally as:

• /content/input_data/NWI_wetlands.gpkg



Reading layer `nwi_wetlands' from data source 
  `/content/input_data/NWI_wetlands.gpkg' using driver `GPKG'
Simple feature collection with 10866 features and 6 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 412977.7 ymin: 2263072 xmax: 820600.7 ymax: 2464465
Projected CRS: WGS 84 / UTM zone 4N


In [9]:
# AOI wetlands for all islands
# https://drive.google.com/file/d/1cKHJ95tAkl0CSoNZkdO0w2wlaH6Wr_YZ/view?usp=drive_link
# drive_download(file = as_id('1cKHJ95tAkl0CSoNZkdO0w2wlaH6Wr_YZ'), path =  '/content/input_data/wetlands.gpgk', overwrite = TRUE)


# AOI wetlands for the islands in UTM 4 (kauai, oahu, molokai, maui)
# https://drive.google.com/file/d/1AsyHxmJ7MUJHwEqXUG61OXQ6bz9mgvDH/view?usp=drive_link
drive_download(file = as_id('1AsyHxmJ7MUJHwEqXUG61OXQ6bz9mgvDH'), path =  '/content/input_data/wetlands.gpgk', overwrite = TRUE)
wetlands_gpkg = st_read('/content/input_data/wetlands.gpgk')

File downloaded:

• aoi_wetland_4.gpkg <id: 1AsyHxmJ7MUJHwEqXUG61OXQ6bz9mgvDH>

Saved locally as:

• /content/input_data/wetlands.gpgk

Warning message in CPL_read_ogr(dsn, layer, query, as.character(options), quiet, :
“GDAL Message 1: File /content/input_data/wetlands.gpgk has GPKG application_id, but non conformant file extension”


Reading layer `aoi_wetland_4' from data source `/content/input_data/wetlands.gpgk' using driver `GPKG'
Simple feature collection with 16 features and 6 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 444046 ymin: 2299914 xmax: 766191.9 ymax: 2457336
Projected CRS: WGS 84 / UTM zone 4N


In [10]:
# NLCD for all islands
# https://drive.google.com/file/d/1foSde5txo6hPvWp4kcQF8We2WTDqDpYh/view?usp=drive_link
drive_download(file = as_id('1foSde5txo6hPvWp4kcQF8We2WTDqDpYh'), path =  '/content/input_data/nlcd_hi_2001', overwrite = TRUE)
nlcd_tif <- raster('/content/input_data/nlcd_hi_2001')

File downloaded:

• nlcd_hi_2001.tif <id: 1foSde5txo6hPvWp4kcQF8We2WTDqDpYh>

Saved locally as:

• /content/input_data/nlcd_hi_2001



In [11]:
download_index_gpkg <- naip_index_gpkg %>% st_intersection(wetlands_gpkg %>% st_transform(st_crs(naip_index_gpkg)))

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”


In [12]:
download_index_gpkg$url %>% unique %>% length

[1] 50

In [14]:
download_index_gpkg %>% data.table

location,url,missionid,id,island,name,lumahai,layer,path,geometry
<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<POLYGON [°]>
m_2215954_se_04_060_20220128.tif,https://coastalimagery.blob.core.windows.net/digitalcoast/HI_NAIP_2021_9668/m_2215954_se_04_060_20220128.tif,9668,1,NA,NA,NA,anahola,E:/Data/AOI_Shapefiles/kauai/anahola.shp,POLYGON ((-159.2891 22.1324...
m_2215954_sw_04_060_20220321.tif,https://coastalimagery.blob.core.windows.net/digitalcoast/HI_NAIP_2021_9668/m_2215954_sw_04_060_20220321.tif,9668,1,NA,NA,NA,anahola,E:/Data/AOI_Shapefiles/kauai/anahola.shp,POLYGON ((-159.3105 22.1323...
m_2115831_ne_04_060_20220211.tif,https://coastalimagery.blob.core.windows.net/digitalcoast/HI_NAIP_2021_9668/m_2115831_ne_04_060_20220211.tif,9668,1,NA,NA,NA,haleiwa,E:/Data/AOI_Shapefiles/oahu/haleiwa.shp,POLYGON ((-158.1268 21.5603...
m_2115831_se_04_060_20220211.tif,https://coastalimagery.blob.core.windows.net/digitalcoast/HI_NAIP_2021_9668/m_2115831_se_04_060_20220211.tif,9668,1,NA,NA,NA,haleiwa,E:/Data/AOI_Shapefiles/oahu/haleiwa.shp,POLYGON ((-158.1226 21.5526...
m_2115832_nw_04_060_20220211.tif,https://coastalimagery.blob.core.windows.net/digitalcoast/HI_NAIP_2021_9668/m_2115832_nw_04_060_20220211.tif,9668,1,NA,NA,NA,haleiwa,E:/Data/AOI_Shapefiles/oahu/haleiwa.shp,POLYGON ((-158.1268 21.5603...
m_2115832_sw_04_060_20220211.tif,https://coastalimagery.blob.core.windows.net/digitalcoast/HI_NAIP_2021_9668/m_2115832_sw_04_060_20220211.tif,9668,1,NA,NA,NA,haleiwa,E:/Data/AOI_Shapefiles/oahu/haleiwa.shp,POLYGON ((-158.0723 21.5523...
m_2215952_ne_04_060_20220128.tif,https://coastalimagery.blob.core.windows.net/digitalcoast/HI_NAIP_2021_9668/m_2215952_ne_04_060_20220128.tif,9668,1,NA,NA,NA,hanalei,E:/Data/AOI_Shapefiles/kauai/hanalei.shp,POLYGON ((-159.5248 22.1857...
m_2215952_se_04_060_20220128.tif,https://coastalimagery.blob.core.windows.net/digitalcoast/HI_NAIP_2021_9668/m_2215952_se_04_060_20220128.tif,9668,1,NA,NA,NA,hanalei,E:/Data/AOI_Shapefiles/kauai/hanalei.shp,POLYGON ((-159.4981 22.1696...
m_2215952_se_04_060_20220321.tif,https://coastalimagery.blob.core.windows.net/digitalcoast/HI_NAIP_2021_9668/m_2215952_se_04_060_20220321.tif,9668,1,NA,NA,NA,hanalei,E:/Data/AOI_Shapefiles/kauai/hanalei.shp,POLYGON ((-159.4981 22.1696...


In [17]:
wetlands_gpkg$layer

[1] "anahola"        "haleiwa"        "hanalei"        "huleia"        
 [5] "kahuku"         "kakahaia"       "kaluaapuhi"     "kamalo"        
 [9] "kanaha"         "kawainui_marsh" "kealia"         "lumahai"       
[13] "paukukalo"      "pearl_harbor"   "waihee"         "wailua_opaekaa"

In [18]:
dir.create('HI_DEM')

In [19]:
#folder link to id
# https://drive.google.com/drive/folders/15nbWHXZdDOZbt9gUuB9BUPeCXc2SoivB?usp=drive_link
jp_folder = "15nbWHXZdDOZbt9gUuB9BUPeCXc2SoivB"
folder_id = drive_get(as_id(jp_folder))
print(folder_id)
#find files in folder
files = drive_ls(folder_id)

#loop dirs and download files inside them
for (i in seq_along(files$name)) {

  i_dir <- files[i, ]
    try({
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/HI_DEM/', i_dir$name[1])
      )
    })
  }


# A dribble: 1 × 3
  name  id                                drive_resource   
  <chr> <drv_id>                          <list>           
1 dem   15nbWHXZdDOZbt9gUuB9BUPeCXc2SoivB <named list [34]>


File downloaded:

• maui_10m.tif <id: 10dHshukbiMC3z7h4xWxeOdY7x6dtkOf2>

Saved locally as:

• /content/HI_DEM/maui_10m.tif

File downloaded:

• kauai_10m.tif <id: 17G_F8uiM6c-0qe24JPCGUZWMAp2uoIro>

Saved locally as:

• /content/HI_DEM/kauai_10m.tif

File downloaded:

• oahu_10m.tif <id: 19_F1B-d5trv8iwBDBNKcLCWa19vnFvGI>

Saved locally as:

• /content/HI_DEM/oahu_10m.tif

File downloaded:

• molokai_10m.tif <id: 14WqA4pNafauzjr2tlJUUJjYROsaH7uEX>

Saved locally as:

• /content/HI_DEM/molokai_10m.tif



In [20]:
# spatial polygon of each dem making the footprints of each dem.
dem_gpkg <-
"/content/HI_DEM" %>%
  list.files(pattern='*.tif', full.names = T) %>%
  # lapply(stack) %>%
  # lapply(extent) %>%
  lapply(function(x){
    x_tif <- x %>% stack
    x_gpkg <- x_tif %>% extent %>% as('SpatialPolygons') %>% st_as_sf(crs = st_crs(x_tif))
    x_gpkg$path <- x
    st_crs(x_gpkg) <- st_crs(x_tif)
    x_gpkg %>% data.table %>% return
    }) %>%
  rbindlist()

In [21]:
dir.create('HI_CCAP')

In [22]:
#folder link to id
# https://drive.google.com/drive/folders/14fLGpMg2n9o6c8YmH4NSHc_ZQOC5MAuv?usp=drive_link
jp_folder = "14fLGpMg2n9o6c8YmH4NSHc_ZQOC5MAuv"
folder_id = drive_get(as_id(jp_folder))
print(folder_id)
#find files in folder
files = drive_ls(folder_id)

#loop dirs and download files inside them
for (i in seq_along(files$name)) {

  i_dir <- files[i, ]
    try({
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/HI_CCAP/', i_dir$name[1])
      )
    })
  }


# A dribble: 1 × 3
  name  id                                drive_resource   
  <chr> <drv_id>                          <list>           
1 ccap  14fLGpMg2n9o6c8YmH4NSHc_ZQOC5MAuv <named list [34]>


File downloaded:

• kauai_ccap_2010.tif <id: 1-9SuvnH-Ck4CMaAXqDQTbbLGKtfMUnxn>

Saved locally as:

• /content/HI_CCAP/kauai_ccap_2010.tif

File downloaded:

• oahu_ccap_2010.tif <id: 1-5Q_qbss1zUnW1wcQlomRHehPemzy7HG>

Saved locally as:

• /content/HI_CCAP/oahu_ccap_2010.tif

File downloaded:

• molokai_ccap_2010.tif <id: 1-2M0JNgYiVMINDoW2mAiWLX3H9kYz_Z->

Saved locally as:

• /content/HI_CCAP/molokai_ccap_2010.tif

File downloaded:

• maui_ccap_2010.tif <id: 1--1nN7Gj_qx6iwPG8Jq5gZcGRA0uaOay>

Saved locally as:

• /content/HI_CCAP/maui_ccap_2010.tif



In [23]:
ccap_gpkg <-
"/content/HI_CCAP" %>%
  list.files(pattern='*.tif', full.names = T) %>%
  # lapply(stack) %>%
  # lapply(extent) %>%
  lapply(function(x){
    x_tif <- x %>% stack
    x_gpkg <- x_tif %>% extent %>% as('SpatialPolygons') %>% st_as_sf(crs = st_crs(x_tif))
    x_gpkg$path <- x
    st_crs(x_gpkg) <- st_crs(x_tif)
    x_gpkg %>% data.table %>% return
    }) %>%
  rbindlist()

In [ ]:
# dem_gpkg %>% st_as_sf %>% st_crs

In [24]:
dir.create('HI_2021_NAIP')

In [28]:
# (runif(1)*10) %>% round

[1] 8

In [25]:
# file.exists('/content/HI_2021_NAIP/m_2015604_se_04_060_20220108.tif')

[1] FALSE

In [ ]:
download_index_gpkg[,] %>% data.table()

In [33]:
cl <- makeCluster(8)


1:nrow(download_index_gpkg) %>%
  parLapply(cl,.,function(x,download_index_gpkg){
  x_dt <- download_index_gpkg[x,]
  download.file(x_dt$url, paste0('/content/HI_2021_NAIP/',x_dt$location,sep='') )
  },download_index_gpkg)


stopCluster(cl)

[[1]]
[1] 0

[[2]]
[1] 0

[[3]]
[1] 0

[[4]]
[1] 0

[[5]]
[1] 0

[[6]]
[1] 0

[[7]]
[1] 0

[[8]]
[1] 0

[[9]]
[1] 0

[[10]]
[1] 0

[[11]]
[1] 0

[[12]]
[1] 0

[[13]]
[1] 0

[[14]]
[1] 0

[[15]]
[1] 0

[[16]]
[1] 0

[[17]]
[1] 0

[[18]]
[1] 0

[[19]]
[1] 0

[[20]]
[1] 0

[[21]]
[1] 0

[[22]]
[1] 0

[[23]]
[1] 0

[[24]]
[1] 0

[[25]]
[1] 0

[[26]]
[1] 0

[[27]]
[1] 0

[[28]]
[1] 0

[[29]]
[1] 0

[[30]]
[1] 0

[[31]]
[1] 0

[[32]]
[1] 0

[[33]]
[1] 0

[[34]]
[1] 0

[[35]]
[1] 0

[[36]]
[1] 0

[[37]]
[1] 0

[[38]]
[1] 0

[[39]]
[1] 0

[[40]]
[1] 0

[[41]]
[1] 0

[[42]]
[1] 0

[[43]]
[1] 0

[[44]]
[1] 0

[[45]]
[1] 0

[[46]]
[1] 0

[[47]]
[1] 0

[[48]]
[1] 0

[[49]]
[1] 0

[[50]]
[1] 0

[[51]]
[1] 0

[[52]]
[1] 0

[[53]]
[1] 0

[[54]]
[1] 0

[[55]]
[1] 0

[[56]]
[1] 0

[[57]]
[1] 0

[[58]]
[1] 0

[[59]]
[1] 0

In [ ]:
# cl <- makeCluster(8)
# 1:nrow(download_index_gpkg) %>%
#   # parLapply(cl,.,function(x,download_index_gpkg){
#   lapply(function(x){
#   library(magrittr)
#   x_dt <- download_index_gpkg[x,]
#   Sys.sleep((runif(1)*10) %>% round)

#   if(!file.exists(paste0('/content/HI_2021_NAIP/',x_dt$location,sep=''))){
#    tryCatch({
#   download.file(
#     paste0("https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/",x_dt$location),
#     # x_dt$url,
#      paste0('/content/HI_2021_NAIP/',x_dt$location,sep='') )
#    },error={return( paste0('/content/HI_2021_NAIP/',x_dt$location,sep=''))})
#   }
#   # },download_index_gpkg)
#   })
# stopCluster(cl)

In [37]:
download.file('https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/HI_NAIP_2021_EPSG-26904.vrt', paste('/content/HI_2021_NAIP/HI_NAIP_2021_EPSG-26904.vrt',sep=''))

In [38]:
hawaii_naip <- stack('/content/HI_2021_NAIP/HI_NAIP_2021_EPSG-26904.vrt')

In [39]:
dir.create('output_fishnets')

In [44]:
hawaii_naip

class      : RasterStack 
dimensions : 480700, 751320, 361159524000, 4  (nrow, ncol, ncell, nlayers)
resolution : 0.6, 0.6  (x, y)
extent     : 364074, 814866, 2172438, 2460858  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=4 +datum=NAD83 +units=m +no_defs 
names      : HI_NAIP_2021_EPSG.26904_1, HI_NAIP_2021_EPSG.26904_2, HI_NAIP_2021_EPSG.26904_3, HI_NAIP_2021_EPSG.26904_4 


In [46]:
wetlands_gpkg

ERROR while rich displaying an object: Error in loadNamespace(x): there is no package called ‘geojsonio’

Traceback:
1. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
2. tryCatchList(expr, classes, parentenv, handlers)
3. tryCatchOne(expr, names, parentenv, handlers[[1L]])
4. doTryCatch(return(expr), name, parentenv, handler)
5. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
6. repr::mime2repr[[mime]](obj)
7. repr_geojson.sf(obj)
8. repr_geojson(geo

id,island,name,lumahai,layer,path,geom
<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<MULTIPOLYGON [m]>
1,NA,NA,NA,anahola,E:/Data/AOI_Shapefiles/kauai/anahola.shp,MULTIPOLYGON (((467439.4 24...
1,NA,NA,NA,haleiwa,E:/Data/AOI_Shapefiles/oahu/haleiwa.shp,MULTIPOLYGON (((590409.2 23...
1,NA,NA,NA,hanalei,E:/Data/AOI_Shapefiles/kauai/hanalei.shp,MULTIPOLYGON (((445906 2451...
1,NA,NA,NA,huleia,E:/Data/AOI_Shapefiles/kauai/huleia.shp,MULTIPOLYGON (((459399.5 24...
1,NA,NA,NA,kahuku,E:/Data/AOI_Shapefiles/oahu/kahuku.shp,MULTIPOLYGON (((602662.2 23...
1,NA,NA,NA,kakahaia,E:/Data/AOI_Shapefiles/molokai/kakahaia.shp,MULTIPOLYGON (((712686.8 23...
1,NA,NA,NA,kaluaapuhi,E:/Data/AOI_Shapefiles/molokai/kaluaapuhi.shp,MULTIPOLYGON (((694186.8 23...
1,NA,NA,NA,kamalo,E:/Data/AOI_Shapefiles/molokai/kamalo.shp,MULTIPOLYGON (((718784.3 23...
1,NA,NA,NA,kanaha,E:/Data/AOI_Shapefiles/maui/kanaha.shp,MULTIPOLYGON (((763679.9 23...


In [47]:
1:nrow(wetlands_gpkg) %>%
  lapply(function(y){
    wetlands_sample <- wetlands_gpkg[y,] %>%
      st_transform(st_crs(hawaii_naip))%>%
      crop(hawaii_naip,.)})

[[1]]
class      : RasterBrick 
dimensions : 3748, 4589, 17199572, 4  (nrow, ncol, ncell, nlayers)
resolution : 0.6, 0.6  (x, y)
extent     : 467437.8, 470191.2, 2447514, 2449763  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=4 +datum=NAD83 +units=m +no_defs 
source     : r_tmp_2024-02-15_034101.268267_950_20929.grd 
names      : HI_NAIP_2021_EPSG.26904_1, HI_NAIP_2021_EPSG.26904_2, HI_NAIP_2021_EPSG.26904_3, HI_NAIP_2021_EPSG.26904_4 
min values :                        26,                        55,                        84,                         1 
max values :                       234,                       228,                       230,                       230 


[[2]]
class      : RasterBrick 
dimensions : 12262, 9419, 115495778, 4  (nrow, ncol, ncell, nlayers)
resolution : 0.6, 0.6  (x, y)
extent     : 590409, 596060.4, 2383570, 2390927  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=4 +datum=NAD83 +units=m +no_defs 
source     : r_tmp_2024-02-15_034116.402

In [95]:
wetlands_fishnet_gpkg <- 1:nrow(wetlands_gpkg) %>%
  lapply(function(y){
  wetlands_fishnet_gpkg <- wetlands_gpkg[y,] %>%
    st_transform(st_crs(hawaii_naip)) %>%
    st_make_grid(cellsize = c(76.8, 76.8)) %>%
    st_as_sf
  })

In [100]:
wetlands_fishnet_gpkg %>% data.table

In [99]:
st_write(wetlands_fishnet_gpkg,paste0('/content/output_fishnets/', wetlands_gpkg$layer[y],'.gpkg'))

ERROR: Error in UseMethod("st_write"): no applicable method for 'st_write' applied to an object of class "list"


In [94]:
1:nrow(wetlands_gpkg) %>%
  lapply(function(y){
  wetlands_fishnet_sample <- wetlands_fishnet_gpkg[y,] %>%
    st_transform(st_crs(hawaii_naip)) %>%
    crop(wetlands_sample,.)
  })

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'crop': object 'wetlands_sample' not found


In [ ]:
# 1:nrow(wetlands_gpkg) %>%
#   lapply(function(y){
#   wetlands_fishnet_gpkg <- wetlands_gpkg[y,] %>%
#     st_transform(st_crs(hawaii_naip)) %>%
#     st_make_grid(cellsize = c(76.8, 76.8)) %>%
#     st_as_sf
#   })

In [ ]:
print(wetlands_gpkg)

In [ ]:
library(dplyr)

In [87]:
wetlands_class <- class(wetlands_gpkg)
print(wetlands_class)

[1] "sf"         "data.frame"


In [86]:
hawaii_naip_class <- class(hawaii_naip)
print(hawaii_naip_class)

[1] "RasterStack"
attr(,"package")
[1] "raster"


In [58]:
# Transform CRS of the entire wetlands dataset to match hawaii_naip
wetlands_transformed <- st_transform(wetlands_gpkg, st_crs(hawaii_naip))

In [ ]:
# Crop the entire wetlands dataset to the extent of hawaii_naip
wetlands_cropped <- st_intersection(wetlands_transformed, hawaii_naip)

In [ ]:
# If hawaii_naip is a raster, convert it to an sf object to use st_intersection for cropping
hawaii_naip_sf <- st_as_sf(as(hawaii_naip, "SpatialPixelsDataFrame"))

In [65]:
install.packages('stars')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘abind’




In [66]:
library(stars)

Loading required package: abind



In [73]:
setwd("/content/HI_2021_NAIP/")

In [91]:
hawaii_naip_index
hawaii_naip

class      : RasterStack 
dimensions : 274360, 542070, 148722325200, 4  (nrow, ncol, ncell, nlayers)
resolution : 0.6, 0.6  (x, y)
extent     : 441786, 767028, 2296194, 2460810  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=4 +datum=NAD83 +units=m +no_defs 
names      : HI_2021_NAIP_index_1, HI_2021_NAIP_index_2, HI_2021_NAIP_index_3, HI_2021_NAIP_index_4 


class      : RasterStack 
dimensions : 480700, 751320, 361159524000, 4  (nrow, ncol, ncell, nlayers)
resolution : 0.6, 0.6  (x, y)
extent     : 364074, 814866, 2172438, 2460858  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=4 +datum=NAD83 +units=m +no_defs 
names      : HI_NAIP_2021_EPSG.26904_1, HI_NAIP_2021_EPSG.26904_2, HI_NAIP_2021_EPSG.26904_3, HI_NAIP_2021_EPSG.26904_4 


In [76]:
# https://drive.google.com/file/d/12X5aI49evFswTAlfAFSvNgPm1lKN7RPE/view?usp=drive_link
drive_download(file = as_id('12X5aI49evFswTAlfAFSvNgPm1lKN7RPE'), path =  '/content/HI_2021_NAIP/HI_2021_NAIP_index.vrt', overwrite = TRUE)
hawaii_naip_index <- stack('/content/HI_2021_NAIP/HI_2021_NAIP_index.vrt')

Auto-refreshing stale OAuth token.

File downloaded:

• HI_2021_NAIP_index.vrt <id: 12X5aI49evFswTAlfAFSvNgPm1lKN7RPE>

Saved locally as:

• /content/HI_2021_NAIP/HI_2021_NAIP_index.vrt



In [74]:
# hawaii_naip <- stack("/content/HI_2021_NAIP/HI_NAIP_2021_EPSG-26904.vrt")

In [78]:
hawaii_naip_indexr <- raster('/content/HI_2021_NAIP/HI_2021_NAIP_index.vrt')

In [ ]:
plot(hawaii_naip_indexr)  # Attempt to plot the first layer to verify it's accessible

In [81]:
# Set the path to the directory containing your .tif files
tif_directory <- "/content/HI_2021_NAIP/" # Adjust this path as necessary

# List all .tif files in the directory
tif_files <- list.files(path = tif_directory, pattern = "\\.tif$", full.names = TRUE)


In [ ]:
# Stack the .tif files
hawaii_naip_stack <- stack(tif_files)

In [83]:
# Load the first raster to use its extent as the target
target_extent <- extent(raster(tif_files[1]))



In [84]:
# Initialize an empty list to hold the adjusted rasters
adjusted_rasters <- list()

# Loop through all tif files, crop them to the target extent, and add to the list
for (tif_file in tif_files) {
  r <- raster(tif_file)
  cropped_r <- crop(r, target_extent)
  adjusted_rasters[[length(adjusted_rasters) + 1]] <- cropped_r
}



ERROR: Error in .local(x, y, ...): extents do not overlap


In [85]:
# Load all rasters
rasters <- lapply(tif_files, raster)

# Extract and print extents
extents <- lapply(rasters, extent)
print(extents)

[[1]]
class      : Extent 
xmin       : 753180 
xmax       : 760194 
ymin       : 2316960 
ymax       : 2324382 

[[2]]
class      : Extent 
xmin       : 753288 
xmax       : 760302 
ymin       : 2310036 
ymax       : 2317458 

[[3]]
class      : Extent 
xmin       : 753288 
xmax       : 760302 
ymin       : 2310036 
ymax       : 2317458 

[[4]]
class      : Extent 
xmin       : 759792 
xmax       : 766806 
ymin       : 2310138 
ymax       : 2317560 

[[5]]
class      : Extent 
xmin       : 759792 
xmax       : 766806 
ymin       : 2310138 
ymax       : 2317560 

[[6]]
class      : Extent 
xmin       : 753498 
xmax       : 760518 
ymin       : 2296194 
ymax       : 2303616 

[[7]]
class      : Extent 
xmin       : 760008 
xmax       : 767028 
ymin       : 2296290 
ymax       : 2303718 

[[8]]
class      : Extent 
xmin       : 714006 
xmax       : 720996 
ymin       : 2330250 
ymax       : 2337660 

[[9]]
class      : Extent 
xmin       : 707514 
xmax       : 714498 
ymin       : 233016

In [72]:
# Check if hawaii_naip is a RasterLayer, RasterBrick, or RasterStack
if (class(hawaii_naip) %in% c("RasterLayer", "RasterBrick", "RasterStack")) {
  # Convert Raster* to SpatialPixelsDataFrame then to an sf object
  hawaii_naip_sf <- st_as_sf(as(hawaii_naip, "SpatialPixelsDataFrame"))
} else {
  # If hawaii_naip is already an sf object, use it directly
  hawaii_naip_sf <- hawaii_naip
}

Warning message in x@cpp$readValues(row - 1, nrows, col - 1, ncols):
“GDAL Error 4: /content/HI_2021_NAIP/./m_2215951_ne_04_060_20220128.tif: No such file or directory”


ERROR: Error: [readValues] cannot read values


In [ ]:
# go through each wetland, one at a time. wetlands_sample is imagery dataset.
# translate dem gpkg

1:nrow(wetlands_gpkg) %>%
  lapply(function(y){
wetlands_sample <- wetlands_gpkg[y,] %>% st_transform(st_crs(hawaii_naip)) %>% crop(hawaii_naip,.)
dem_loc_gpkg <- dem_gpkg %>% st_as_sf %>% st_transform(st_crs(wetlands_gpkg[y,])) %>% st_intersection(wetlands_gpkg[y,])
dem_load_tif <- dem_loc_gpkg$path %>% raster %>% crop(., wetlands_fishnet_gpkg[y,] %>% st_transform(st_crs(dem_load_tif)))
nlcd_load_tif <- nlcd_tif %>% crop(., wetlands_fishnet_gpkg[y,] %>% st_transform(st_crs(nlcd_tif)))
wetlands_fishnet_gpkg <- wetlands_gpkg[y,] %>% st_transform(st_crs(hawaii_naip)) %>% st_make_grid(cellsize =c(76.8,76.8)) %>% st_as_sf
# file.remove(paste0('/content/output_fishnets/',wetlands_sample$layer[1],'.gpkg'))
st_write(wetlands_fishnet_gpkg,paste0('/content/output_fishnets/',wetlands_gpkg$layer[y],'.gpkg'),delete_dsn=T,delete_layer = T)

cl <- makeCluster(8)
1:nrow(download_index_gpkg) %>%
  parLapply(cl,.,function(x,wetlands_fishnet_gpkg,hawaii_naip,wetlands_sample,dem_load_tif,nlcd_load_tif,nwi_gpkg){

library(raster)
library(sf)
library(fasterize)
library(magrittr)
library(googledrive)
library(parallel)
library(data.table)
sample_chip <- wetlands_fishnet_gpkg[x,] %>% st_transform(st_crs(hawaii_naip)) %>% crop(wetlands_sample,.)
sample_chip$dem <- projectRaster(dem_load_tif, sample_chip, method="ngb")
sample_chip$nlcd <- projectRaster(nlcd_load_tif, sample_chip, method="ngb")
nwi_gpkg$wetland_ty <- nwi_gpkg$wetland_ty %>% as.factor
nwi_gpkg <- nwi_gpkg %>% st_transform(st_crs(sample_chip))
sample_chip$class <- fasterize(
nwi_gpkg,
sample_chip[[1]],
field = 'wetland_ty',
fun = "first")
sample_chip$class_fill <-sample_chip$class
values(sample_chip$class_fill)[is.na(values(sample_chip$class_fill))] <- values(sample_chip$nlcd)[is.na(values(sample_chip$class_fill))]
values(sample_chip$class_fill)[is.na(values(sample_chip$class_fill))] <- 0
values(sample_chip$dem)[is.na(values(sample_chip$dem))] <- 0
values(sample_chip$nlcd)[is.na(values(sample_chip$nlcd))] <- 0
values(sample_chip$class)[is.na(values(sample_chip$class))] <- 0
sample_chip  %>% writeRaster(paste0('output_chips/',(sample_chip %>% extent %>% round)[1],'_',(sample_chip %>% extent %>% round)[3],'.tif'),overwrite=T)

  },wetlands_fishnet_gpkg,hawaii_naip,wetlands_sample,dem_load_tif,nlcd_load_tif,nwi_gpkg)
stopCluster(cl)
  })

Zip files and save to drive

In [35]:
# zip(zipfile = 'testZip', files = 'testDir/test.csv') #make this work
zip(zipfile = "/content/HI_2021_NAIP_4islands.zip",
    files = c("/content/HI_2021_NAIP"))

In [36]:
#put code here to upload to drive
# save to https://drive.google.com/drive/folders/1rTT6ZCyV8gxQBZYt6kPVZ331ya5kP-aT?usp=drive_link
drive_upload(media = "/content/HI_2021_NAIP_4islands.zip", path = as_id("1rTT6ZCyV8gxQBZYt6kPVZ331ya5kP-aT"))

Local file:

• /content/HI_2021_NAIP_4islands.zip

Uploaded into Drive file:

• HI_2021_NAIP_4islands.zip <id: 1Ehz62pSNL26DOxEWN2czXIegAJwQ80kZ>

With MIME type:

• application/zip

